<a href="https://colab.research.google.com/github/deviknitkkr/ML-Models/blob/main/Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
from prophet import Prophet

# Read stock data from yfinance for last 20 months
stock = yf.Ticker('SBIN.NS')
df = stock.history(period="15mo")

fig = go.Figure()
fig.add_trace(go.Candlestick(x=df.index,open=df['Open'], high=df['High'],low=df['Low'], close=df['Close']))
fig.add_trace(go.Scatter(x=df.index, y=df['Close'].rolling(window=5).mean(), name='SMA 1',mode='lines', line=dict(color='gray')))
fig.update_layout(title='Stock Price', yaxis_title='Price')
fig.show()

In [7]:

df_prophet = df.reset_index()[['Date', 'Close']].rename({'Date':'ds', 'Close':'y'}, axis=1)
df_prophet['ds'] = pd.to_datetime(df_prophet['ds']).dt.tz_localize(None) # Remove timezone

# Exclude Saturdays and Sundays
df_prophet = df_prophet[df_prophet['ds'].dt.weekday < 5]

model = Prophet(changepoint_prior_scale=0.05,
                holidays_prior_scale=5,
                seasonality_prior_scale=10,
                changepoint_range=.999,
                daily_seasonality=False,
                weekly_seasonality=True,
                yearly_seasonality=True)
model.add_country_holidays(country_name='IN')
model.fit(df_prophet)

future = model.make_future_dataframe(periods=100, freq='D')
future = future[future['ds'].dt.weekday < 5]

forecast = model.predict(future)

/usr/local/lib/python3.9/dist-packages/prophet/hdays.py:209: Warning:

We only support Diwali and Holi holidays from 2010 to 2030

DEBUG:cmdstanpy:input tempfile: /tmp/tmps8y78m85/du093l4c.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmps8y78m85/6ncqwxq7.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=4993', 'data', 'file=/tmp/tmps8y78m85/du093l4c.json', 'init=/tmp/tmps8y78m85/6ncqwxq7.json', 'output', 'file=/tmp/tmps8y78m85/prophet_model1vlnqq6y/prophet_model-20230329153848.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:38:48 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:38:48 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.9/dist-packages/prophet/hdays.py:209: Warning:

We only support Diwali and Holi holidays f

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_prophet['ds'], y=df_prophet['y'], name='Actual'))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted'))
fig.update_layout(title='Stock Price Prediction', yaxis_title='Price', xaxis_title='Date')

# Mark dotted straight line on today
fig.update_layout(shapes=[dict(
                type= 'line',
                xref= 'x', yref= 'paper',
                x0= df_prophet['ds'].iloc[-1], y0= 0,
                x1= df_prophet['ds'].iloc[-1], y1= 1,
                line=dict(color='Gray', width=2, dash='dot'))])
fig.show()